# 4.4 word2vecに関する残りのテーマ

- word2vec の仕組みや実装については、ほとんど説明は終わりました。本節ではword2vec について、これまで扱いきれなかったテーマを紹介したいと思います。

## 4.4.1 word2vecを使ったアプリケーションの例

- 大きなコーパス（WikipediaやGoogle News のテキストデータなど）で学習
<br>↓単語の分散表現を活用
- テキスト分類や文書クラスタリング、品詞タグ付け、感情分析などに活用

- 自然言語をベクトルに変換できれば、一般的な機械学習の手法（ニューラルネットワークやSVMなど）が適用可能。 そのベクトルを別の機械学習システムの入力とすることができる
- 単語の分散表現の学習と機械学習システムの学習は別のデータセットを使い、個別に学習を行うのが一般的
    - 例
        - 単語の分散表現については、Wikipedia のような汎用的なコーパスを使って学習
        - 現在直面している問題に対して集められたデータを対象に、機械学習システム（SVM など）の学習
        - ただし、現在直面する問題の学習データが大量に存在するのであれば、単語の分散表現と機械学習システムの学習を同時にゼロから行うことも考えられる
![4_4](fig4/4_4.png)

#### メールの自動分類システム

- 前提
    - 今あなたは、利用者が1 億人を超えるスマートフォンアプリの開発・運営をしているとします。
    - 会社には、手に負えないほどのメールがユーザーから毎日届きます（または、Twitter などで多くのつぶやきを見つけることができます）。
    - 好意的な意見がある一方で、中には、不満を持つユーザーの声も存在するでしょう。

- やりたいこと
    - 送られてくるメール（そして“つぶやき” など）を自動で分類できるシステムを作れないかと考えます。
    - たとえば、図4-22 のように、メールの内容からユーザーの感情を3段階に分類することはできないかと考えます。
    - ユーザーの感情を正しく分類することができれば、不満を持つユーザーのメールから順に目を通すことができます。そうすれば、アプリの致命的な問題を発見し、早期に手を打つことができるかもしれません。
![4_22](fig4/4_22.png)

- 実現方法
    - メールの自動分類システムを作成するには、まずはデータ（メール）を収集することから始めます。
    - ここでの例ではユーザーから送られてきたメールを集め、そのメールに対して人手でラベル付けを行います。
    - たとえば、3 段階の感情を表すラベル――positive / neutral / negative ――を付与します。
    - そのラベル付けの作業が終わったら、学習済みのword2vec を用いてメールをベクトルへ変換します。
    - 後は、感情分析を行う何らかの分類システム（SVM やニューラルネットワークなど）に対して、ベクトル化されたメールと感情ラベルを与えて学習させるのです。
    - この例のように、自然言語を扱う問題は、単語の分散表現によってベクトルに変換することができます。それによって、通常の機械学習の手法で解くことができるようになります。さらに、word2vec の転移学習の恩恵が得られます。つまり、自然言語の多くのタスクでは、word2vec の単語の分散表現を利用することで精度の向上が期待できるのです。

## 4.4.2 単語ベクトルの評価方法

- 上記のアプリケーションにおける課題
    - 単語の分散表現を作るシステム（word2vec）と、特定の問題に対して分類を行うシステム（たとえば感情を分類するSVM など）が別の場合、単語の分散表現の次元数が最終的な精度にどのように影響するかを調べるには、
        - まず単語の分散表現の学習を行い、
        - そしてその分散表現を使って、もうひとつの機械学習システムの学習が必要
    - →つまり、2 段階の学習を行った上で評価する必要がある。

        - さらにその場合、2 つのシステムにおいて最適なハイパーパラメータのためのチューニングも必要になり、多くの時間がかかってしまいます。

- 解決法
    - 単語の分散表現の良さを評価するにあたり、現実的なアプリケーションとは切り離して評価を行うというのが一般的によく行われます。
    - その際によく用いられる評価指標が単語の「類似性」や「類推問題」による評価です。
        - 単語の類似性の評価
            - 人間が作成した単語類似度の評価セットを使って評価することが多く行われます。たとえば、0 から10 の間でスコア化するとして、「cat」と「animal」の類似度は8、「cat」と「car」の類似度は2 といったように、人が単語間の類似性を採点します）。そして、人が出したスコアとword2vec によるコサイン類似度のスコアを比較して、その相関性を見るのです。
        - 類推問題による評価
            - 「king : queen = man : ?」のような類推問題を出題し、その正解率でもって単語の分散表現の良さを測ります。たとえば、論文[27] では、類推問題による評価結果が掲載されています。ここでは、その結果から一部を抜粋して、図4-23 に示します。
![4_23](fig4/4_23.png)
- 図4-23 では、word2vec のモデル、単語の分散表現の次元数、そしてコーパスのサイズをパラメータとして、比較実験を行っています。
    - 「Semantics」
        - 単語の意味を類推する類推問題への正解率を示します。これはたとえば、「king : queen = actor : actress」のような単語の意味を問う問題です。
    - 「Syntax」
        - 単語の形態情報を問う問題で、たとえば「bad: worst = good : best」のような問題に相当します。

        
- 図4-23 の結果から、次のことが分かります。
    - モデルによって精度が異なる（コーパスに応じて最適なモデルを選ぶ）
    - コーパスが大きいほど良い結果になる（ビックデータは常に望まれる）
    - 単語ベクトルの次元数は適度な大きさが必要（大きすぎても精度が悪くなる）

# 4.5 まとめ

- 本章では、word2vec の高速化をテーマに、前章のCBOWモデルに対して改良を加えました。
- 具体的には、Embedding レイヤを実装しNegative Sampling という新しい手法を導入しました。この背景には、コーパスの語彙数が増えるのに比例し、計算量が増加することが挙げられます。

- 本章での重要なテーマは、「すべて」ではなく「一部」を処理することです。結局のところ、人間がすべてを知ることができないように、コンピュータも（現在の性能では）すべてのデータを処理することは現実的ではありません。それよりも、自分にとって大切な少数のことに限定して取り組むほうが実りが多いのです。本章ではその考えに基づく手法―― Negative sampling――を詳しく見てきました。Negativesampling は、「すべて」の単語ではなく、「一部」の単語だけを対象にすることで、計算の効率化を達成しました。

- 前章と本章にて、word2vec をテーマとする一連の話は終わりです。word2vec は自然言語の分野に多大な影響を与えてきました。そこで得られた単語の分散表現は、さまざまな自然言語処理のタスクに利用されています。さらにword2vec の思想は、自然言語だけではなく他の分野――音声、画像、動画など――にも応用されています。本章でword2vec をしっかりと理解したのであれば、その知識はさまざまな分野で役に立つはずです。